In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle
from tqdm import tqdm
import re
from keras.preprocessing import text, sequence

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train = pd.read_pickle(FILE.train_ori.value)
train_length = len(train)
test = pd.read_pickle(FILE.test_ori.value)
X = pd.concat([train,test],sort=False)
X = X.reset_index(drop=True)

# Process Model

In [3]:


def get_tok(X,col,train_length,d_filter='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    X[col] = X[col].astype(str)
    tok_dict_all = {}
    tok_dict_train = {}
    tok_dict_test = {}
    tok_all=text.Tokenizer(num_words=X[col].nunique(),lower=False,filters=d_filter)
    tok_all.fit_on_texts(list(X[col].values))
    print(len(tok_all.word_index))

    tok_train=text.Tokenizer(num_words=X[col].iloc[:train_length].nunique(),lower=False,filters=d_filter)
    tok_train.fit_on_texts(list(X[col].iloc[:train_length].values))

    tok_test=text.Tokenizer(num_words=X[col].iloc[train_length:].nunique(),lower=False,filters=d_filter)
    tok_test.fit_on_texts(list(X[col].iloc[train_length:].values))
    word_intersection = set(tok_train.word_index.keys()).intersection(set(tok_test.word_index.keys()))
    print(len(word_intersection))

    self_index = {}
    count = 1
    for word in word_intersection:
        self_index[word] = count
        count+=1
    self_index['unknown'] = count

    for word in tok_all.word_index.keys():
        tok_all.word_index[word] = self_index.get(word,count)
    return tok_all


In [4]:
tok = get_tok(X,col='user_tags',train_length=train_length,d_filter=',')

1405
1312


In [5]:
tok = get_tok(X,col='model',train_length=train_length,d_filter=',')

13507
3320


# Demo

In [50]:
sentence = ['I like eating apple','apple is good']
tok = text.Tokenizer(num_words=15,lower=False)
tok.fit_on_texts(sentence)
print(tok.word_index)
tok.word_index['apple'] = 11
print(tok.texts_to_sequences(sentence))

{'like': 3, 'eating': 4, 'is': 5, 'I': 2, 'good': 6, 'apple': 1}
[[2, 3, 4, 11], [11, 5, 6]]
